In [5]:
import cv2
import os
import numpy as np
import tensorflow as tf

# Cargar los modelos de detección de puntos clave faciales y reconocimiento de emociones
with open('detection.json', 'r') as json_file:
    json_savedModel = json_file.read()

model_1_facialKeyPoints = tf.keras.models.model_from_json(json_savedModel)
model_1_facialKeyPoints.load_weights('weights_keypoint.hdf5')
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model_1_facialKeyPoints.compile(loss="mean_squared_error", optimizer=adam, metrics=['accuracy'])

with open('emotion.json', 'r') as json_file:
    json_savedModel = json_file.read()

model_2_emotion = tf.keras.models.model_from_json(json_savedModel)
model_2_emotion.load_weights('weights_emotions.hdf5')
model_2_emotion.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

dataPath = 'data'  # Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=', imagePaths)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()
    nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    faces = faceClassif.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        rostro = auxFrame[y:y + h, x:x + w]
        rostro_resized = cv2.resize(rostro, (96, 96), interpolation=cv2.INTER_CUBIC)

        # Detección de puntos clave faciales
        # Aquí llamamos al modelo 1 para detectar los puntos clave faciales en la imagen del rostro
        keypoints = model_1_facialKeyPoints.predict(np.expand_dims(rostro_resized, axis=0))
        print(keypoints)
        # Reconocimiento de emociones
        # Aquí llamamos al modelo 2 para reconocer las emociones asociadas a los puntos clave faciales
        emotion_prediction = model_2_emotion.predict(np.expand_dims(rostro_resized, axis=0))
        # print(emotion_prediction)
        # Clasificar la emoción a partir de la predicción
        emotion_class = np.argmax(emotion_prediction)
        # print(emotion_prediction)

        # Mostrar la emoción predicha en la imagen
        emotions_labels = {0: 'Ira', 1: 'Odio', 2: 'Tristeza', 3: 'Felicidad', 4: 'Sorpresa'}
        predicted_emotion = emotions_labels[emotion_class]
        cv2.putText(frame, predicted_emotion, (x, y - 5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)

        # Dibujar el rectángulo alrededor del rostro
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Mostrar la imagen con la emoción en tiempo real
        nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    cv2.imshow('nFrame', nFrame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()


imagePaths= []
1/1 [==============================] - 0s 480ms/step
[[ 9019.998   5663.199   4057.6824  6151.951   8183.6978  5822.5054
  10005.19    5726.4077  5130.703   6127.011   3026.5757  6404.1113
   7893.206   4618.4424 10892.134   4534.6597  6029.5327  4803.9053
   1758.9572  5488.153   8001.5674  8801.27    9261.696  12447.756
   5167.8604 12898.47    7488.7534 11757.915   7668.5083 12836.402 ]]
1/1 [==============================] - 0s 46ms/step
[[ 9107.4375  5712.184   4097.1787  6184.5366  8261.946   5867.0537
  10098.876   5776.6226  5161.927   6160.9214  3071.7805  6427.987
   7934.558   4631.3643 11028.206   4580.7544  6054.085   4814.3267
   1792.8627  5492.737   7950.452   8888.621   9275.239  12564.194
   5176.4097 12995.287   7469.3354 11843.264   7650.778  12945.556 ]]
1/1 [==============================] - 0s 33ms/step
[[ 6998.636   4393.3174  3184.4731  4798.4517  6345.902   4526.365
   7750.8174  4435.5713  3997.4492  4776.8364  2392.2546  4996.052
   6096.2876 

# P   R   U   E   B   A   S

In [53]:
import cv2
import os
import numpy as np
import tensorflow as tf
import pandas as pd

# Cargar los modelos de detección de puntos clave faciales y reconocimiento de emociones
with open('detection.json', 'r') as json_file:
    json_savedModel = json_file.read()

model_1_facialKeyPoints = tf.keras.models.model_from_json(json_savedModel)
model_1_facialKeyPoints.load_weights('weights_keypoint.hdf5')
adam = tf.keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model_1_facialKeyPoints.compile(loss="mean_squared_error", optimizer=adam, metrics=['accuracy'])

with open('emotion.json', 'r') as json_file:
    json_savedModel = json_file.read()

model_2_emotion = tf.keras.models.model_from_json(json_savedModel)
model_2_emotion.load_weights('weights_emotions.hdf5')
model_2_emotion.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

dataPath = 'data'  # Cambia a la ruta donde hayas almacenado Data
imagePaths = os.listdir(dataPath)
print('imagePaths=', imagePaths)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# cap.set(3, 480)
# cap.set(4, 300)

faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Crear una lista para almacenar las predicciones de emociones en varios cuadros
emotion_predictions_buffer = []

# Crear un DataFrame para almacenar las emociones predichas
df_emociones = pd.DataFrame(columns=['Emocion'])

while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    auxFrame = gray.copy()
    nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    faces = faceClassif.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        # Obtener la región de interés (ROI) de la cara para detectar puntos clave faciales
        face_roi = gray[y:y + h, x:x + w]
        face_roi_resized = cv2.resize(face_roi, (96, 96), interpolation=cv2.INTER_CUBIC)
        face_roi_normalized = face_roi_resized / 255.0
        face_roi_reshaped = np.reshape(face_roi_normalized, (1, 96, 96, 1))

        # Detección de puntos clave faciales
        keypoints = model_1_facialKeyPoints.predict(face_roi_reshaped)

        # Reconocimiento de emociones
        emotion_prediction = model_2_emotion.predict(face_roi_reshaped)
        emotion_class = np.argmax(emotion_prediction)

        # Almacena la predicción en el buffer de predicciones
        emotion_predictions_buffer.append(emotion_class)

        # Limita el tamaño del buffer a un número determinado de cuadros
        buffer_size = 10
        if len(emotion_predictions_buffer) > buffer_size:
            emotion_predictions_buffer.pop(0)

        # Realiza el promedio de las predicciones en el buffer
        average_emotion_prediction = np.mean(emotion_predictions_buffer)

        # Si la diferencia entre la predicción actual y el promedio es menor que un umbral, muestra la predicción promediada
        threshold = 2
        if len(set(emotion_predictions_buffer)) == 1 or abs(emotion_class - average_emotion_prediction) < threshold:
            predicted_emotion = emotions_labels[int(round(average_emotion_prediction))]
        else:
            predicted_emotion = emotions_labels[emotion_class]

        # Almacenar las emociones detectadas

        df_emociones = df_emociones.concat({'Emocion': predicted_emotion}, ignore_index=True)

        # Mostrar la emoción predicha en la imagen
        cv2.putText(frame, predicted_emotion, (x, y - 5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)

        # Dibujar el rectángulo alrededor del rostro
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Mostrar la imagen con la emoción en tiempo real
        nFrame = cv2.hconcat([frame, np.zeros((480, 300, 3), dtype=np.uint8)])

    cv2.imshow('nFrame', nFrame)
    k = cv2.waitKey(1)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

df_emociones.unique()


imagePaths= []
1/1 [==============================] - 1s 505ms/step


AttributeError: 'DataFrame' object has no attribute 'concat'

: 

In [52]:
import pandas as pd
import json

# Supongamos que tienes el DataFrame original df con la columna 'Emocion'

# Calcular la frecuencia de cada emoción
emotion_counts = df_emociones['Emocion'].value_counts()

# Calcular el total de emociones
total_emotions = len(df_emociones)

# Calcular el porcentaje de cada emoción
# emotion_percentages = emotion_counts


# Crear el nuevo DataFrame
df_emotion_summary = pd.DataFrame({
    'Emocion': emotion_counts.index,
    'Cantidad': emotion_counts.values
})

# Ordenar el DataFrame por tipo de emoción (opcional)
df_emotion_summary = df_emotion_summary.sort_values(by='Emocion')

# Restablecer los índices del DataFrame
df_emotion_summary = df_emotion_summary.reset_index(drop=True)

# Exportar el DataFrame a un archivo JSON
df_emotion_summary.to_json('emotion_summary.json', orient='records', indent=2)

print(df_emotion_summary)


     Emocion  Cantidad
0  Felicidad        70
1        Ira         8
2       Odio        26
3   Tristeza        50
